In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = 'NousResearch/Llama-2-7b-hf'

tokenizer = AutoTokenizer.from_pretrained(model_id)
target_model = AutoModelForCausalLM.from_pretrained(model_id)
supervisor_model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import EvalPrediction

from typing import Dict

training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy='epoch')

def compute_metrics(eval_pred: EvalPrediction) -> Dict[str, float]:
    predictions, _ = eval_pred
    # TODO: measure alignment using `supervisor_model`
    return {}

trainer = Trainer(
    model=target_model,
    args=TrainingArguments,
    train_dataset=None, # TODO: TruthfulQA
    eval_dataset=None, # TODO: TruthfulQA
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()